In [2]:
from dotenv import load_dotenv
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import (
    FewShotChatMessagePromptTemplate,
    ChatPromptTemplate
)
from langchain_core.output_parsers import StrOutputParser

# .env 파일을 불러와서 환경 변수로 설정
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# 모델 초기화
llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",  # 원하는 모델 사용
    temperature=0.7
)

# 1. 예시 데이터 (examples)
# 뉴스 기사와 그에 대한 키워드 예시를 3개 이상 포함
examples = [
    {
        "news": "삼성전자가 내년 초에 자체적으로 개발한 인공지능(AI) 가속기를 처음으로 출시할 예정이다. 이는 AI 반도체 시장에서 지배적인 위치를 차지하고 있는 엔비디아의 독점을 도전하고, 세계 최고의 반도체 제조업체로서의 지위를 다시 확립하려는 삼성전자의 노력으로 해석된다.",
        "keywords": "삼성전자, 인공지능, 엔비디아"
    },
    {
        "news": "세계보건기구(WHO)는 최근 새로운 건강 위기에 대응하기 위해 국제 협력의 중요성을 강조했다. 전염병 대응 역량의 강화와 글로벌 보건 시스템의 개선이 필요하다고 발표했다.",
        "keywords": "세계보건기구, 건강위기, 국제협력"
    },
    {
        "news": "최근 한류 콘텐츠가 전 세계적으로 큰 인기를 얻고 있다. 특히 K팝과 한국 드라마는 미국, 유럽 등지에서 젊은 세대에게 폭발적인 반응을 이끌어내며 문화 수출의 새로운 지평을 열고 있다.",
        "keywords": "한류, K팝, 한국드라마"
    },
    {
        "news": "서울시가 시민들의 편의를 위해 대중교통 노선을 대폭 확충하고, 심야 버스 운행을 확대하기로 했다. 이와 함께 공공 자전거 '따릉이'의 이용자 맞춤형 서비스를 도입하여 교통 효율성을 높일 계획이다.",
        "keywords": "서울시, 대중교통, 따릉이"
    }
]

# 2. 예시 프롬프트 (대화형)
# FewShotPromptTemplate이 참고할 각 예시의 입력-출력 형식을 정의
example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{news}"),
    ("ai", "키워드: {keywords}")
])

# 3. Few-Shot 프롬프트
# 위에서 정의한 예시들을 모아 Few-Shot 템플릿을 생성
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples
)

# 4. 최종 프롬프트 (시스템 메시지 포함)
# Few-Shot 프롬프트와 사용자 입력 프롬프트를 결합하여 최종 프롬프트를 만듦
final_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 뉴스 기사에서 핵심 키워드 3개를 추출하는 전문가입니다. 사용자의 요청에 대해 항상 '키워드: '로 시작하는 형식으로 답변하세요."),
    few_shot_prompt,
    ("human", "{news}")
])

# 5. LLM Chain 연결 및 실행
# 프롬프트, 언어 모델, 출력 파서를 LCEL로 연결
chain = final_prompt | llm | StrOutputParser()

# 테스트 뉴스
test_news = "제미나이 2.0 플래시는 현재 구글 AI 스튜디오(Google AI Studio) 및 버텍스 AI(Vertex AI)에서 제미나이 API를 통해 개발자에게 실험 모델로 제공됩니다. 모든 개발자는 멀티모달 입력 및 텍스트 출력을 사용할 수 있으며, 텍스트 음성 변환(text-to-speech) 및 네이티브 이미지 생성은 일부 파트너들을 대상으로 제공됩니다. 내년 1월에는 더 많은 모델 사이즈와 함께 일반에 공개될 예정입니다."

# 체인 실행 및 결과 출력
result = chain.invoke({"news": test_news})
print(result)

키워드: 제미나이, 구글, AI
